In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import re
import os



In [ ]:
# url='https://gorkhapatraonline.com/categories/national'

# response= requests.get(url)
# soup = BeautifulSoup(response.content, 'html.parser')
# print(soup.prettify())


In [ ]:
urls =[
        'https://gorkhapatraonline.com/',
        # 'https://gorkhapatraonline.com/categories/national'
        ]

data = {
        'शीर्षक': [],
        'विवरण': [],
        }
df = pd.DataFrame(data)
df
      

In [ ]:


for url in urls:
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        soup = BeautifulSoup(response.content, 'html.parser')
        
        article = soup.find_all('section', class_='article-section')
        
        links = []
        for a in article:
            try:
                l = a.find_all('a')
                for link_element in l:
                    href = link_element.get('href')
                    if href:  # Only add non-empty hrefs
                        links.append(href)
            except Exception as error:
                print(f"Error extracting links from article section: {error}")
                continue
        # Remove duplicates by converting the list to a set and back to a list
        links = list(set(links))
        # Filter out links that do not contain 'news'
        links = [link for link in links if 'news' in link]  
        # Filter out links that are not HTTP or HTTPS
        links = [link for link in links if link.startswith(('http://', 'https://'))]

        # Print the number of links found
        print(f"Found {len(links)} links in {url}")
        
        for link in links:
            try:
                # Handle relative URLs if needed
                if link.startswith('/'):
                    # Assuming you need to prepend base URL
                    # link = base_url + link  # Uncomment and set base_url if needed
                    pass
                
                response = requests.get(link, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                
                section = soup.find_all('section', class_='article-section')
                
                for sec in section:
                    try:
                        # Check if h1 exists before accessing text
                        h1_element = sec.find('h1')
                        if h1_element is None:
                            print(f"No h1 found in section for link: {link}")
                            continue
                            
                        title = h1_element.text.strip()
                        title = re.sub(r'[a-zA-Z0-9/.-]', '', title)
                        
                        print(title)
                        
                        des = sec.find_all('div', class_="blog-details")
                        description = ''
                        
                        for desc in des:
                            try:
                                para = desc.find_all('p')
                                for p in para:
                                    text = p.text.strip()
                                    description += text + ' '
                            except Exception as e:
                                print(f"Error extracting paragraph text: {e}")
                                continue
                        
                        # Clean up the description
                        description = re.sub(r'\uf040', '', description)
                        description = re.sub(r'[a-zA-Z0-9/.-]', '', description)

                        description= '"' + description + '"'
                        
                        # Only add to dataframe if we have both title and description
                        if title and description.strip():
                            df.loc[len(df)] = [title, description]
                        
                    except Exception as e:
                        print(f"Error processing section: {e}")
                        continue
                        
            except requests.exceptions.RequestException as e:
                print(f"Request error for link {link}: {e}")
                continue
            except Exception as e:
                print(f"Unexpected error processing link {link}: {e}")
                continue
    
    except requests.exceptions.RequestException as e:
        print(f"Request error for URL {url}: {e}")
        continue
    except Exception as e:
        print(f"Unexpected error processing URL {url}: {e}")
        continue

# Save to CSV (moved outside the loop to save once after processing all URLs)
try:
    today = date.today().strftime("%Y-%m-%d")
    foldername = '/var/home/ramrshrcg/Desktop/Python/scrapping/News_csv/gorkhapatra'
    
    # Create directory if it doesn't exist
    os.makedirs(foldername, exist_ok=True)
    
    file_path = f"{foldername}/{today}_gorkhapatra_news.csv"
    
    if os.path.exists(file_path):
        df.to_csv(file_path, mode='a', index=False, header=False)
    else:
        df.to_csv(file_path, index=False)
        
    print(f"Data saved to {file_path}")
    
except Exception as e:
    print(f"Error saving to CSV: {e}")

In [ ]:
df